In [4]:
import pandas as pd
import numpy as np
import os
import json
import csv

TL_sentence_path = r'C:\Users\edcrf\sentence_dataTL.csv'
VL_sentence_path = r'C:\Users\edcrf\sentence_dataVL.csv'

# data파일 불러오기
TL_sentence_data = pd.read_csv(TL_sentence_path, encoding='utf-8')
VL_sentence_data = pd.read_csv(VL_sentence_path, encoding='utf-8')

# 중복 제거, Pronuncication 열은 필요 없다고 생각
TL_sentence_data.drop('Pronunciation', axis=1, inplace=True)
TL_sentence_data = TL_sentence_data.drop_duplicates().reset_index(drop=True)
VL_sentence_data.drop('Pronunciation', axis=1, inplace=True)
VL_sentence_data = VL_sentence_data.drop_duplicates().reset_index(drop=True)

In [5]:
# 형태소 분석기로 Okt를 사용
import konlpy
from konlpy.tag import Kkma, Komoran, Okt, Hannanum
okt = Okt()

standard_sentences = TL_sentence_data['Standard']
dialect_sentences = TL_sentence_data['Dialect']

In [68]:
from tqdm import tqdm

# 방언 문장과 표준어 문장 각각 형태소 분석 진행
standard_okt = []
dialect_okt = []

for i in tqdm(range(0, len(TL_sentence_data))):
    standard_okt.append(okt.morphs(standard_sentences[i]))
    dialect_okt.append(okt.morphs(dialect_sentences[i]))

100%|██████████| 323371/323371 [28:20<00:00, 190.18it/s]


In [79]:
# 토큰화된 방언 문장과 표준어 문장을 각각 csv파일로 저장
with open('st_okt.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(standard_okt)

with open('di_okt.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(dialect_okt)

In [74]:
# # csv저장을 위해 리스트를 DataFrame으로 변환
# standard_okt_df = pd.DataFrame(standard_okt)
# dialect_okt_df = pd.DataFrame(dialect_okt)

# standard_okt_df.to_csv(r'C:\Users\edcrf\standard_okt.csv', index=False, sep =',')
# dialect_okt_df.to_csv(r'C:\Users\edcrf\dialect_okt.csv', index=False, sep =',')

In [ ]:
# # csv파일로 저장한 토큰화된 결과 불러오기
# standard_okt_data = pd.read_csv(r'C:\Users\edcrf\standard_okt.csv')
# dialect_okt_data = pd.read_csv(r'C:\Users\edcrf\dialect_okt.csv')

In [8]:
# 저장된 토큰화 데이터 불러오기
standard_okt_data = []
dialect_okt_data = []

with open('st_okt.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        standard_okt_data.append(row)

with open('di_okt.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        dialect_okt_data.append(row)